In [9]:
# Install dependencies
# cmd -> "pip install flask flask-api"

from flask_api import FlaskAPI
from flask import request, jsonify, Response
import keras
#keras.__version__
import numpy as np
from numpy import genfromtxt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
from keras.optimizers import RMSprop
from keras.models import load_model
import os

In [10]:
# Define any values here so that we can easily change for the competition
PORT = 5000
AI_NAME = "Mike Hancho"

In [11]:
model = load_model('KJKaggleConv2D.h5')

In [12]:
def whoami(prev_moves):
    return 1 if len(prev_moves) % 2 == 0 else -1

def switchPlayer(player):
    player = player *-1
    return player

def convertGameStateToBoardState(moves):
    board = np.zeros(42)
    board = board.astype('float32')
    column_counts = np.array([6,6,6,6,6,6,6])
    player = 1

    for move in moves:
        move = int(move)
        position = (7 * column_counts[move - 1]) - (7 - move)
        column_counts[move - 1] -= 1
        board[position - 1] = player
        player = switchPlayer(player)

    return board

In [13]:

# This is an example of the function that will take the input of the previous moves and determine the next move
# Instead of rando here, plug in your model!
#####################################################################################################################
##### HINT: You will want to convert the input to something more meaningful before providing it to your model #######
#####################################################################################################################
def determineNextMove(prev_moves):
    # Split input into an array and convert each character to an int
    moves = [int(move) for move in prev_moves]

    # Create an array with a list of valid moves
    valid_moves = []

    # Check each column to see if it is full of pieces
    # Add each valid move to the list of candidate moves
    for candidate_move in range(1, 8):
        if moves.count(candidate_move) < 6:
            valid_moves.append(candidate_move)

    # Get the number of possible valid moves
    num_of_possible_valid_moves = len(valid_moves)
          
    iam = whoami(prev_moves)
    highest_move = 0
    highest_prediction = 2 if iam == -1 else -2
    
    for candidate_move in valid_moves:
        test_input = np.append(moves, candidate_move)
        test_board_state = convertGameStateToBoardState(test_input)
        prediction = model.predict(test_board_state.reshape(1,6,7,1))[0][0]
        if (iam == 1 and prediction > highest_prediction or iam == -1 and prediction < highest_prediction):
            highest_move = candidate_move
            highest_prediction = prediction
    # Return the result
    return highest_move

In [15]:
# Create a web server
app = FlaskAPI(__name__)

# To Test this route run
# curl -X GET http://127.0.0.1:5000/api/nextmove\?prevmoves\="34331"
@app.route('/api/nextmove', methods=['Get'])
def nextmove():
    prev_moves = request.args.get('prevmoves')
    next_move = determineNextMove(prev_moves)

    resp = jsonify(next_move)
    resp.headers['Access-Control-Allow-Origin'] = '*'
    return resp

@app.route('/api/name', methods=['Get'])
def name():
    resp = jsonify(AI_NAME)
    resp.headers['Access-Control-Allow-Origin'] = '*'
    return resp

if __name__ == "__main__":
    app.run(port=PORT)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/May/2019 19:38:48] "GET /api/nextmove?prevmoves= HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:38:58] "GET /api/nextmove?prevmoves=3 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:39:16] "GET /api/nextmove?prevmoves=1 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:39:21] "GET /api/nextmove?prevmoves=141 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:39:25] "GET /api/nextmove?prevmoves=14156 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:39:27] "GET /api/nextmove?prevmoves=1415651 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:39:33] "GET /api/nextmove?prevmoves=141565115 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:39:37] "GET /api/nextmove?prevmoves=14156511546 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:39:40] "GET /api/nextmove?prevmoves=1415651154662 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:39:50] "GET /api/nextmove?prevmoves=141565115466221 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:40:05] "GET /ap

127.0.0.1 - - [10/May/2019 19:47:19] "GET /api/nextmove?prevmoves=333345645442216555246664223 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:47:24] "GET /api/nextmove?prevmoves=33334564544221655524666422332 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:47:27] "GET /api/nextmove?prevmoves=3333456454422165552466642233256 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:47:29] "GET /api/nextmove?prevmoves=333345645442216555246664223325677 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:49:01] "GET /api/nextmove?prevmoves= HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:49:01] "GET /api/nextmove?prevmoves=4 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:49:01] "GET /api/nextmove?prevmoves=44 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:49:01] "GET /api/nextmove?prevmoves=444 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:49:01] "GET /api/nextmove?prevmoves=4444 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:49:02] "GET /api/nextmove?prevmoves=44445 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:49:02

127.0.0.1 - - [10/May/2019 19:52:49] "GET /api/nextmove?prevmoves=44443256335 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:52:56] "GET /api/nextmove?prevmoves=4444325633521 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:52:57] "GET /api/nextmove?prevmoves=444432563352122 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:53:10] "GET /api/nextmove?prevmoves=44443256335212256 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:53:13] "GET /api/nextmove?prevmoves=4444325633521225655 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:53:32] "GET /api/nextmove?prevmoves=444432563352122565546 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:53:35] "GET /api/nextmove?prevmoves=44443256335212256554636 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:53:40] "GET /api/nextmove?prevmoves=4 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:53:42] "GET /api/nextmove?prevmoves=444 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:53:43] "GET /api/nextmove?prevmoves=44444 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:53:45] "GET 

127.0.0.1 - - [10/May/2019 19:55:14] "GET /api/nextmove?prevmoves=444456 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:55:14] "GET /api/nextmove?prevmoves=4444562 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:55:15] "GET /api/nextmove?prevmoves=44445623 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:55:15] "GET /api/nextmove?prevmoves=444456233 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:55:15] "GET /api/nextmove?prevmoves=4444562333 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:55:15] "GET /api/nextmove?prevmoves=44445623333 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:55:15] "GET /api/nextmove?prevmoves=444456233332 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:55:15] "GET /api/nextmove?prevmoves=4444562333321 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:55:15] "GET /api/nextmove?prevmoves=44445623333216 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:55:16] "GET /api/nextmove?prevmoves=444456233332165 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 19:55:16] "GET /api/nextmove?prevmoves

...

127.0.0.1 - - [10/May/2019 21:24:51] "GET /api/nextmove?prevmoves=444456233332165 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:24:51] "GET /api/nextmove?prevmoves=4444562333321656 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:24:51] "GET /api/nextmove?prevmoves=44445623333216566 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:24:52] "GET /api/nextmove?prevmoves=444456233332165665 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:25:06] "GET /api/nextmove?prevmoves=4 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:25:08] "GET /api/nextmove?prevmoves=444 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:25:10] "GET /api/nextmove?prevmoves=44444 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:25:11] "GET /api/nextmove?prevmoves=4444423 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:25:13] "GET /api/nextmove?prevmoves=444442321 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:25:14] "GET /api/nextmove?prevmoves=44444232131 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:25:16] "GET /api/nextmove?prevmoves=44

127.0.0.1 - - [10/May/2019 21:33:38] "GET /api/nextmove?prevmoves=44445623 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:33:38] "GET /api/nextmove?prevmoves=444456233 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:33:38] "GET /api/nextmove?prevmoves=4444562333 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:33:38] "GET /api/nextmove?prevmoves=44445623333 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:33:38] "GET /api/nextmove?prevmoves=444456233332 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:33:39] "GET /api/nextmove?prevmoves=4444562333321 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:33:39] "GET /api/nextmove?prevmoves=44445623333216 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:33:39] "GET /api/nextmove?prevmoves=444456233332165 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:33:39] "GET /api/nextmove?prevmoves=4444562333321656 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:33:39] "GET /api/nextmove?prevmoves=44445623333216566 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:33:39] "GET /ap

127.0.0.1 - - [10/May/2019 21:34:35] "GET /api/nextmove?prevmoves=6543213342563 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:34:35] "GET /api/nextmove?prevmoves=65432133425634 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:34:35] "GET /api/nextmove?prevmoves=654321334256345 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:34:38] "GET /api/nextmove?prevmoves=654321 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:34:38] "GET /api/nextmove?prevmoves=6543213 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:34:38] "GET /api/nextmove?prevmoves=65432133 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:34:38] "GET /api/nextmove?prevmoves=654321334 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:34:39] "GET /api/nextmove?prevmoves=6543213342 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:34:39] "GET /api/nextmove?prevmoves=65432133425 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:34:39] "GET /api/nextmove?prevmoves=654321334256 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:34:39] "GET /api/nextmove?prevmoves

127.0.0.1 - - [10/May/2019 21:35:07] "GET /api/nextmove?prevmoves=77744442135332 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:35:07] "GET /api/nextmove?prevmoves=777444421353322 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:35:07] "GET /api/nextmove?prevmoves=7774444213533222 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:35:08] "GET /api/nextmove?prevmoves=77744442135332223 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:35:08] "GET /api/nextmove?prevmoves=777444421353322234 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:35:08] "GET /api/nextmove?prevmoves=7774444213533222343 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:35:08] "GET /api/nextmove?prevmoves=77744442135332223433 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:35:08] "GET /api/nextmove?prevmoves=777444421353322234331 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:35:08] "GET /api/nextmove?prevmoves=7774444213533222343314 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:35:09] "GET /api/nextmove?prevmoves=77744442135332223433141 H

127.0.0.1 - - [10/May/2019 21:36:18] "GET /api/nextmove?prevmoves=415 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:36:18] "GET /api/nextmove?prevmoves=4156 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:36:18] "GET /api/nextmove?prevmoves=41562 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:36:19] "GET /api/nextmove?prevmoves=415623 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:36:19] "GET /api/nextmove?prevmoves=4156232 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:36:19] "GET /api/nextmove?prevmoves=41562322 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:36:19] "GET /api/nextmove?prevmoves=415623224 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:36:19] "GET /api/nextmove?prevmoves=4156232243 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:36:19] "GET /api/nextmove?prevmoves=41562322433 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:36:19] "GET /api/nextmove?prevmoves=415623224332 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2019 21:36:20] "GET /api/nextmove?prevmoves=4156232243324 HTTP/1.1" 200 -

In [14]:
determineNextMove('4')

4